<a href="https://colab.research.google.com/github/Nobobi-Hasan/Fine_Tune_Llama/blob/main/FineTune_Llama_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-Tune with Llama Project** ***(This is an ongoing test project)

In [ ]:
# !pip install torchtune

In [ ]:
# !tune ls

RECIPE                                   CONFIG                                  
full_finetune_single_device              llama2/7B_full_low_memory               
                                         code_llama2/7B_full_low_memory          
                                         llama3/8B_full_single_device            
                                         llama3_1/8B_full_single_device          
                                         llama3_2/1B_full_single_device          
                                         llama3_2/3B_full_single_device          
                                         mistral/7B_full_low_memory              
                                         phi3/mini_full_low_memory               
                                         phi4/14B_full_low_memory                
                                         qwen2/7B_full_single_device             
                                         qwen2/0.5B_full_single_device           
                

In [ ]:
# !huggingface-cli download meta-llama/Llama-3.2-1B-Instruct --local-dir /tmp/Llama-3.2-1B-Instruct

In [ ]:
# !torchtune run full_finetune_single_device --config llama3_2/1B_full_single_device device=cpu epochs=1
!tune run full_finetune_single_device --config llama3_2/1B_full_single_device device=cpu epochs=1

INFO:torchtune.utils._logging:Running FullFinetuneRecipeSingleDevice with resolved config:

batch_size: 4
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Llama-3.2-1B-Instruct/
  checkpoint_files:
  - model.safetensors
  model_type: LLAMA3_2
  output_dir: /tmp/torchtune/llama3_2_1B/full_single_device
  recipe_checkpoint: null
clip_grad_norm: null
compile: false
dataset:
  _component_: torchtune.datasets.alpaca_dataset
  packed: false
device: cpu
dtype: bf16
enable_activation_checkpointing: false
enable_activation_offloading: false
epochs: 1
gradient_accumulation_steps: 1
log_every_n_steps: 1
log_peak_memory_stats: true
loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss
max_steps_per_epoch: null
metric_logger:
  _component_: torchtune.training.metric_logging.DiskLogger
  log_dir: /tmp/torchtune/llama3_2_1B/full_single_device/logs
model:
  _component_: torchtune.models.llama3_2.llama3_2_1b
optimizer:
  _component_: bitsandby

In [ ]:
from datasets import load_dataset, Dataset
ds = load_dataset(
  'bitext/Bitext-customer-support-llm-chatbot-training-dataset',
  split="train"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Bitext_Sample_Customer_Support_Training_(…):   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

In [ ]:
print(ds.shape)

(26872, 5)


In [ ]:
print(ds.column_names)

['flags', 'instruction', 'category', 'intent', 'response']


In [ ]:
import pprint
pprint.pprint(ds[0])

{'category': 'ORDER',
 'flags': 'B',
 'instruction': 'question about cancelling order {{Order Number}}',
 'intent': 'cancel_order',
 'response': "I've understood you have a question regarding canceling order "
             "{{Order Number}}, and I'm here to provide you with the "
             'information you need. Please go ahead and ask your question, and '
             "I'll do my best to assist you."}


In [ ]:
ds[1]

{'flags': 'BQZ',
 'instruction': 'i have a question about cancelling oorder {{Order Number}}',
 'category': 'ORDER',
 'intent': 'cancel_order',
 'response': "I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority."}

In [ ]:
first_thousand_points = ds[:1000]
ds = Dataset.from_dict(first_thousand_points)

In [ ]:
print(ds.shape)

(1000, 5)


In [ ]:
def merge_example(row):
  row['conversation'] = f"Query: {row['instruction']}\nResponse: {row['response']}"
  return row

In [ ]:
ds = ds.map(merge_example)
print(ds[0]['conversation'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Query: question about cancelling order {{Order Number}}
Response: I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.


In [ ]:
print(ds.shape)

(1000, 6)


In [ ]:
ds[1]

{'flags': 'BQZ',
 'instruction': 'i have a question about cancelling oorder {{Order Number}}',
 'category': 'ORDER',
 'intent': 'cancel_order',
 'response': "I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority.",
 'conversation': "Query: i have a question about cancelling oorder {{Order Number}}\nResponse: I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority."}

In [ ]:
ds.save_to_disk("preprocessed_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
ds_preprocessed = load_from_disk("preprocessed_dataset")

# **Chapter 2**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name="Maykeye/TinyLLama-v0"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/9.25M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
# Load the dataset
dataset = load_dataset(
    "bitext/Bitext-customer-support-llm-chatbot-training-dataset",
    split="train"
)


In [ ]:
from transformers import TrainingArguments
training_arguments = TrainingArguments(
  	# Set learning rate
    learning_rate=2e-3,
    warmup_ratio=0.03,
  	num_train_epochs=3,
  	output_dir='/tmp',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    save_steps=10,
    logging_steps=2,
    lr_scheduler_type='constant',
    report_to='none'
)

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 33.0 MB/s eta 0:00:00


In [ ]:
# Import the supervised fine-tuning class
from trl import SFTTrainer, SFTConfig

sft_config = SFTConfig(
    dataset_text_field="conversation",
    # can also move other arguments here, like learning_rate
)

# Instantiate fine-tuning class
trainer = SFTTrainer(
    # Pass necessary arguments
    model=model,
    # tokenizer=tokenizer,
    processing_class=tokenizer,
    train_dataset=dataset,
    args=training_arguments,
    **sft_config.to_dict()
)

# Start training
trainer.train()

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'output_dir'

**Evaluate generated text using ROUGE**
You are given 10 samples from a question-answering dataset (Softage-AI/sft-conversational_dataset).

You have used TinyLlama-1.1B to generate answers to these samples, and your task is to evaluate the quality of the generated results to the ground truth.

The answers generated by this model are provided in test_answers and the ground truth in reference_answers. Use the ROUGE evaluation metrics to evaluate the quality of the model's generation.

**Instructions**
100 XP
* Import evaluation class and metric (ROUGE metric).
* Instantiate the evaluation class and load the ROUGE metric.
* Run the evaluator instance with the given reference_answers and test_answers to compute the ROUGE scores.
* Store, in final_score the score from the results that checks overlap of word pairs between the reference and generated answers.

In [ ]:
# !pip install evaluate
# !pip install rouge_score

In [ ]:
# Import the evaluation library from Hugging Face
import evaluate

# Instantiate your evaluate library and load the ROUGE metric
rouge_evaluator = evaluate.load('rouge')

# Fill in the method, and place your reference answers and test answers
results = rouge_evaluator.compute(predictions = test_answers, references = reference_answers)

# Extract the ROUGE1 score from the results dictionary
final_score = results['rouge1']
print(final_score)

### LoRA using PEFT

In [ ]:
from peft import LoraConfig

In [ ]:
# Instantiate LoRA configuration with values
lora_config = LoraConfig(
  	r=12,
    lora_alpha=12,
  	task_type="CAUSAL_LM",
    lora_dropout=0.05,
    bias="none",
    target_modules=['q_proj', 'v_proj']
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_arguments,
  	# Pass the lora_config to trainer
  	peft_config = lora_config,,
)

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    # Set rank parameter
  	r=2,
  	# Set scaling factor
    lora_alpha=4,
  	# Set the type of task
  	task_type='CAUSAL_LM',
    lora_dropout=0.05,
    bias="none",
    target_modules=['q_proj', 'v_proj']
)

training_arguments = TrainingArguments(
    max_seq_length=250, # ***here
    dataset_text_field='conversation', # ***here
  	# Set learning rate
    learning_rate=2e-3,
    warmup_ratio=0.03,
  	num_train_epochs=3,
  	output_dir='/tmp',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    save_steps=10,
    logging_steps=2,
    lr_scheduler_type='constant',
    report_to='none'
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    # max_seq_length=250, # Added to training_arguments
    # dataset_text_field='conversation', # Added to training_arguments
    tokenizer=tokenizer,
    args=training_arguments,
  	peft_config=peft_config,
)

trainer.train()

### **Quantization**

In [1]:
!pip install bitsandbytes
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_use_double_quant=True, # extra from auto fill
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# !pip install -U bitsandbytes
# !pip install -U bitsandbytes --force-reinstall

model = AutoModelForCausalLM.from_pretrained(
    "nvidia/Llama3-ChatQA-1.5-8B",
    quantization_config=bnb_config
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.08G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-8B")

promptstr = """System: You are a helpful chatbot who answers questions about planets.
User: Explain the history of Mars
Assistant: """

inputs = tokenizer.encode(promptstr, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_length=200)
decoded_outputs = tokenizer.decode(outputs[0, inputs.shape[1]:], skip_special_tokens = True)
print("\nOutput:", decoded_outputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Output:  There is not much history of Mars, as it is a planet and not a civilization. However, it has been observed and studied for centuries.


In [ ]:
# Finetuning a quantized model

trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    train_dataset=ds,
    max_seq_length=250,
    dataset_text_field='conversation',
    tokenizer=tokenizer,
    args=training_arguments
)
trainer.train()